# Neural Network 정의
nn_id : Neural Network ID<br>
biz_cate : Business 대분류<br>
biz_sub_cate : Business 소분류<br>
nn_title : Neural Network Title<br>
nn_desc : Neural Network Description<br>
use_flag : 사용여부<br> 
dir : 문제유형<br>
config : Custom 여부 

In [4]:
##### import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "w2v0005"
nn_wf_ver_id ="1"

# Seq - 1
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("1.evaluation result : {0}".format(data))

1.evaluation result : w2v0005


# Work Flow Version 정의
nn_def_list_info_nn_id :<br>
nn_wf_ver_info : Work Flow Version Description<br>
condition :<br>
active_flag : 사용여부
master_nn_ver_wflist_info insert

In [5]:
# Seq - 2
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("2.evaluation result : {0}".format(data))

2.evaluation result : w2v0005


# Work Flow Node  Initialize
type : Neural Network Type<br>
1. Data Node<br>
2. Neural Network Node<br>
3. Evaluation Node<br>

In [6]:
# Seq - 3
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/'+ nn_id + '/wfver/1/',
                     json={
                         "type": "word2vec"
                     })
data = json.loads(resp.json())
print("3.evaluation result : {0}".format(data))

3.evaluation result : word2vec


# Data Node Config
type : Image Source Type<br>
labels : Classfication Labels<br>
source_path : Image Source Path<br>
preprocess : Image Preprocess Config<br>
store_path : Image Store Path

In [7]:
# Seq - 4
return_dict = {}
return_dict['test'] = open('../../data/novel_1.txt', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("4.evaluation result : {0}".format(data))

# Seq - 5
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+ nn_id + '/ver/1/node/data_node/',
                     json={
                         "source_server": "local",
                         "source_sql": "all",
                         "max_sentence_len" : 10
                     })
data = json.loads(resp.json())
print("5.evaluation result : {0}".format(data))


4.evaluation result : ['1 file upload success']
5.evaluation result : {'source_server': 'local', 'max_sentence_len': 10, 'source_parse_type': 'raw', 'source_path': '/hoya_src_root/w2v0005/1/data_node', 'source_type': 'local', 'source_sql': 'all'}


# Select Preprocess Type
preprocess : kkma, twiter, mecab, nltk<br>

In [8]:
# Seq - 6
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/pre/nnid/'+ nn_id + '/ver/1/node/data_node/',
                     json={
                         "preprocess":  "mecab",
                     })
data = json.loads(resp.json())
print("6.evaluation result : {0}".format(data))

# Seq - 7
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/store/nnid/'+ nn_id + '/ver/1/node/data_node/')
data = json.loads(resp.json())
print("7.evaluation result : {0}".format(data))

# Seq - 8
resp = requests.get('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+ nn_id + '/ver/1/node/data_node/')
data = json.loads(resp.json())
print("8.evaluation result : {0}".format(data))

6.evaluation result : mecab
7.evaluation result : /hoya_str_root/w2v0005/1/data_node
8.evaluation result : {'source_server': 'local', 'source_path': '/hoya_src_root/w2v0005/1/data_node', 'preprocess': 'mecab', 'source_parse_type': 'raw', 'max_sentence_len': 10, 'source_type': 'local', 'store_path': '/hoya_str_root/w2v0005/1/data_node', 'source_sql': 'all'}


# Neural Network Node Config
model_path : Model Save Path<br>
window_size : 문장 내의 현재 단어와 예측 단어 사이의 최대 거리.<br>
vector_size : 백터사이즈.<br>
batch_size : Batch Size (Vocabary Distance)<br>
iter : Trainning Iteration<br>

In [9]:
# Seq - 9
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/w2v/nnid/' + nn_id + '/ver/' + nn_wf_ver_id + '/node/netconf_node/',
                     json={
                        "window_size" : 5,
                        "vector_size" : 100,
                        "batch_size" : 100,
                        "iter" : 5,
                        "min_count" : 1
                     })
data = json.loads(resp.json())
print("9.evaluation result : {0}".format(data))

9.evaluation result : {'vector_size': 100, 'window_size': 5, 'model_path': '/hoya_model_root/w2v0005/1/netconf_node', 'batch_size': 100, 'min_count': 1, 'iter': 5}


# Eval Network Node Config

In [10]:
# Seq - 10
return_dict = {}
return_dict['test'] = open('../../data/novel_1.txt', 'rb')
resp = requests.post('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+ nn_id + '/ver/1/node/test_data_node/',
                     files = return_dict)
data = json.loads(resp.json())
print("10.evaluation result : {0}".format(data))

# Seq - 11
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+ nn_id + '/ver/1/node/test_data_node/',
                     json={
                         "source_server": "local",
                         "source_sql": "all",
                         "max_sentence_len" : 50
                     })
data = json.loads(resp.json())
print("11.evaluation result : {0}".format(data))

# Seq - 12
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/pre/nnid/'+ nn_id + '/ver/1/node/test_data_node/',
                     json={
                         "preprocess":  "mecab",
                     })
data = json.loads(resp.json())
print("12.evaluation result : {0}".format(data))

# Seq - 13
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/store/nnid/'+ nn_id + '/ver/1/node/test_data_node/')
data = json.loads(resp.json())
print("13.evaluation result : {0}".format(data))

node_name = 'eval_node'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/eval/nnid/'+nn_id+'/ver/1/node/eval_node/',
                    json={
                        "type": "w2v",
                    })
data = json.loads(resp.json())
print("14.evaluation result : {0}".format(data))

10.evaluation result : ['1 file upload success']
11.evaluation result : {'source_server': 'local', 'max_sentence_len': 50, 'source_parse_type': 'raw', 'source_path': '/hoya_src_root/w2v0005/1/test_data_node', 'source_type': 'local', 'source_sql': 'all'}
12.evaluation result : mecab
13.evaluation result : /hoya_str_root/w2v0005/1/test_data_node
14.evaluation result : {'type': 'w2v'}


# Training

In [11]:
# Seq - 14
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+nn_wf_ver_id+'/')
data = json.loads(resp.json())
print("14.evaluation result : {0}".format(data))

14.evaluation result : [None, None, None, None, True, {'x': [], 'y': [], 'word': []}]


# Prediction
type :  sim
val_1 : [철강,권오준,연구원,포스코]


포스코-적자
철강-포스코


In [12]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/w2v/nnid/' + nn_id + '/ver/active/',
                     json={
                         "type": "vector",
                         "val_1":["김승우", "이미지"],
                         "val_2":[]
                        }
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [[0.01969406194984913, -0.09804179519414902, 0.1542636752128601, 0.2112734019756317, -0.037264686077833176, 0.10920611768960953, -0.1596326380968094, -0.023802446201443672, -0.020082926377654076, -0.16287624835968018, 0.11105943471193314, 0.12063600867986679, 0.055013664066791534, -0.0608241893351078, 0.003094926942139864, 0.09242644160985947, 0.04621683061122894, 0.03490554168820381, 0.05245642736554146, 0.06112836301326752, 0.06081978231668472, 0.10848554223775864, -0.09712579846382141, 0.05378593131899834, 0.1518940031528473, 0.1037895530462265, -0.1779768317937851, 0.03627336397767067, -0.04067004844546318, 0.12469716370105743, 0.11134537309408188, -0.03336101397871971, 0.06870795786380768, -0.06295514851808548, 0.15352946519851685, -0.11334399878978729, 0.18121683597564697, -0.07570145279169083, 0.18824899196624756, -0.027422551065683365, 0.013871768489480019, 0.05743801221251488, -0.06971363723278046, -0.08156920224428177, 0.016776004806160927, 0.1171353384852